# 💾 Acceso a Azure Data Lake Storage (ADLS Gen2) desde Databricks

Este diagrama ilustra cómo **Databricks** se conecta y accede a los datos almacenados en **Azure Data Lake Storage Gen2 (ADLS Gen2)**.

---

## 💻 Componentes Principales

* **Databricks:** La plataforma de análisis.
    * **Cluster:** El recurso computacional que ejecuta las tareas.
    * **Notebook:** El entorno donde escribes y ejecutas el código (por ejemplo, Python o SQL).
* **Azure:** El proveedor de la nube.
    * **Storage Account:** El contenedor que aloja los datos.
    * **ADLS Gen2:** La capa de almacenamiento optimizada para Big Data.

---

## 🔑 Métodos de Autenticación y Seguridad

Para que el Cluster/Notebook de Databricks pueda leer o escribir en ADLS Gen2, necesita autenticarse. Los métodos de autenticación comunes son:

### 1. Service Principal (Microsoft Entra ID/Azure Active Directory) 🛡️

* Es el método **más recomendado** y seguro para entornos de producción.
* Se utiliza una identidad de aplicación (el **Service Principal**) registrada en **Microsoft Entra ID** (anteriormente Azure AD) para obtener acceso.

### 2. Storage Access Keys 🔑

* Claves maestras que otorgan **acceso completo** a la cuenta de almacenamiento.
* **No son recomendadas** para acceso de producción debido a que si se filtran, comprometen toda la cuenta.

### 3. Shared Access Signature (SAS Token) 🔗

* Tokens temporales que otorgan acceso **limitado** (por tiempo, permisos y recursos específicos) a los datos.



####Storage Access Keys 
Para acceder a crear un secrectScope en databricks dirigite
al home de databricks y agrega en la url esto: #secrets/createScope
1. Establecer la configuración de spark "fs.azure.account.key"
2. Listar archivos del contenedor "bronze"
3. Leer datos del archivo "categorias.csv"

In [0]:
spark.conf.set(
    "fs.azure.account.key.adlsg2udemycourse.dfs.core.windows.net",
    dbutils.secrets.get(scope="sc-proyect-joseph", key="key-adlsg2udemycourse"))

In [0]:
#Sin nosabemos el nombre del scope
dbutils.secrets.listScopes()

In [0]:
# Para listar tus secretos en adlsg
dbutils.secrets.list("sc-proyect-joseph")

In [0]:
# PARA LISTAR LOS ARCHIVOS QUE TENEMOS EN EL STORAGE ACCOUNT
display(
dbutils.fs.ls("abfss://bronze@adlsg2udemycourse.dfs.core.windows.net/"))

In [0]:
df_categorias=spark.read.csv("abfss://bronze@adlsg2udemycourse.dfs.core.windows.net/categorias.csv",header=True).limit(5).display()


####Token SAS(SHARED ACCESS SIGNATURE)
Para acceder a crear un secrectScope en databricks dirigite
al home de databricks y agrega en la url esto: #secrets/createScope
1. Establecer la configuración de spark "fs.azure.account.key"
2. Listar archivos del contenedor "bronze"
3. Leer datos del archivo "categorias.csv"

In [0]:
spark.conf.set("fs.azure.account.auth.type.adlsg2proyectudemy.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.adlsg2proyectudemy.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.adlsg2proyectudemy.dfs.core.windows.net", dbutils.secrets.get(scope="sc-udemy-course", key="secrect-sas"))

In [0]:
#ESTO ES PAREA VISUALIZAR LA LISTA DE SCOPES CREADOS EN DATABRICKS
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list("sc-udemy-course")

In [0]:
df_categorias=spark.read.csv("abfss://bronze@adlsg2proyectudemy.dfs.core.windows.net/categorias.csv",header=True).limit(5).display()

## 🔐 Acceso Seguro a Azure Data Lake Gen2 desde Databricks (Service Principal)

Este documento detalla los pasos y el flujo de trabajo para configurar el acceso seguro a **Azure Data Lake Storage Gen2 (ADLS Gen2)** desde un clúster de **Azure Databricks** utilizando un **Service Principal (Entidad de Servicio)** registrado en Microsoft Entra ID (antes Azure Active Directory) y el control de acceso basado en roles (**RBAC**).

---

## 🏗️ Flujo de Acceso

El diagrama ilustra el flujo de autenticación y autorización:

* **Databricks** necesita acceder a los datos almacenados en **ADLS Gen2** (que se representa con los iconos de SQL, Data Lakes y otros servicios).
* La autenticación se realiza a través de **Microsoft Entra ID**. Databricks utiliza una identidad registrada:
    * **User Account** (Cuenta de Usuario) o, más comúnmente para servicios automatizados,
    * **Service Principal** (Entidad de Servicio), que actúa como una identidad no interactiva para la aplicación.
* El **Service Principal** debe tener asignado un **Rol de RBAC** (Role-Based Access Control) a nivel del **Resource Group** o la cuenta de almacenamiento de ADLS Gen2 para poder acceder.
* El rol asignado (**Owner**, **Contributor**, **Storage Blob Data Contributor**, **Custom**, etc.) determina los permisos que el Service Principal tendrá sobre los datos y la infraestructura.

---

## 🛠️ Pasos de Configuración Requeridos

Para implementar esta conexión de forma segura, se deben seguir los siguientes pasos clave:

### 1. Preparación en Microsoft Entra ID (Azure Portal)

* **Registrar la Aplicación en Microsoft Entra ID / Server Principal**: Se crea un registro de aplicación. Esto genera un **Application ID (Client ID)** y un **Directory ID (Tenant ID)** únicos.
* **Generar un secreto (contraseña) para la Aplicación**: Se crea un secreto de cliente asociado a la aplicación registrada. Este secreto es la "contraseña" que Databricks usará para autenticarse.

### 2. Autorización (RBAC)

* **Asignar el Rol "Storage Blob Data Contributor" al Data Lake**: Este es el rol más común y con el **mínimo privilegio** necesario para leer, escribir y eliminar datos en contenedores y blobs de ADLS Gen2. Este rol se asigna al **Service Principal** en el alcance de la cuenta de almacenamiento o un contenedor específico.

### 3. Configuración en Databricks (Spark)

* **Configurar Spark con App / Client Id, Directory / Tenant Id & Secret**: El código de Spark o la configuración del clúster de Databricks debe recibir las credenciales del Service Principal para poder autenticarse. Esto se hace típicamente estableciendo propiedades de configuración de Spark en el notebook o el clúster.

In [0]:
#obteniendo credenciales como tenan id , client id
client_id="2a547da5-f769-4d90-83af-145052c0496a"
tenant_id="b824ae35-fbf9-4eaa-b4da-79ef1f190582"


In [0]:
dbutils.secrets.listScopes()

In [0]:
#Esto te da los secretos creados en azure key vault
dbutils.secrets.list("sc-udemy-course")

In [0]:
client_secret= dbutils.secrets.get(scope="sc-udemy-course",key="client-secret-udemy")

spark.conf.set("fs.azure.account.auth.type.adlsg2proyectudemy.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsg2proyectudemy.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsg2proyectudemy.dfs.core.windows.net", f"{client_id}")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsg2proyectudemy.dfs.core.windows.net",client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsg2proyectudemy.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
df_categorias=spark.read.csv("abfss://bronze@adlsg2proyectudemy.dfs.core.windows.net/categorias.csv",header=True).limit(5).display()

## 🔑 Autenticación Centralizada: Ámbito de Clúster (Cluster-Scoped Authentication)

Este método es para que los usuarios accedan a datos en **Azure Data Lake Storage Gen2 (ADLS Gen2)** de forma segura, canalizando las credenciales de autenticación a través de la configuración de un clúster de Azure Databricks.

Este enfoque asegura que las credenciales **no estén expuestas directamente en el código del notebook**, sino que se gestionen a nivel de la infraestructura de cómputo.

---

### 🏗️ Flujo de Acceso

* **User (Usuario)**: El usuario interactúa con un **Notebook** de Databricks.
* **Notebook**: El código dentro del notebook (por ejemplo, PySpark o Scala) contiene comandos de lectura/escritura de datos, pero **no contiene las credenciales sensibles** (como las Access Keys o los Service Principal Secrets).
* **Cluster (Clúster)**: El notebook ejecuta el código sobre un **Clúster** de Databricks. La clave de este método es que el clúster está configurado con las propiedades de autenticación.
    * **Secrets**: Las credenciales reales (generalmente un Service Principal ID y su Secreto, o un OAuth Token) se almacenan de forma segura en **Databricks Secret Scopes**.
    * Estas propiedades se inyectan en la **Configuración de Spark** del clúster antes de que comience a ejecutarse el código del usuario.
* **ADLS Gen2**: Cuando el código del notebook intenta acceder a la ruta de ADLS Gen2, el clúster utiliza las credenciales preconfiguradas en su ámbito para autenticarse y acceder a los datos.